# Problema Continuo (Hiperelasticidad Casi Incompresible $u-p$)

Sea $\Omega \subset \mathbb{R}^d$ con borde $\partial\Omega = \Gamma_D \cup \Gamma_N$.

* **Desplazamiento:** $u:\Omega\to\mathbb{R}^d$.
* **Presión:** $p:\Omega\to\mathbb{R}$.

## Deformación
Las cantidades de deformación se definen como:
$$
F=I+\nabla u, \quad J=\det F, \quad C=F^\top F, \quad I_c=\operatorname{tr}(C).
$$

## Energía y Tensiones
**Energía (parte desviadora Neo-Hooke + acoplo volumétrico con $p$):**
$$
\psi_{\mathrm{dev}}(F)=\tfrac{\mu}{2}\big(I_c-2-2\ln J\big).
$$

**Primer tensor de Piola:**
$$
P_{\mathrm{dev}}=\frac{\partial \psi_{\mathrm{dev}}}{\partial F},\qquad 
P=P_{\mathrm{dev}}+p\,F^{-\top}.
$$

**Restricción volumétrica relajada:**
$$
\phi(u,p):=\ln J-\frac{p}{\lambda}=0.
$$

## Ecuaciones en $\Omega$
Ecuaciones en $\Omega$ con tracciones $t$ en $\Gamma_N$:
$$
\begin{aligned}
\operatorname{Div} P &= 0, \quad & \text{en }\Omega \\
\ln J-\tfrac{p}{\lambda} &= 0, \quad & \text{en }\Omega \\
u&=\bar u, \quad & \text{en }\Gamma_D \\
P\,n&=t, \quad & \text{en }\Gamma_N
\end{aligned}
$$

## Formulación Débil

**Espacios:**
$$
\mathcal{V}=\{v\in [H^1(\Omega)]^d: v=0\text{ en }\Gamma_D\},\qquad 
\mathcal{Q}=L^2(\Omega).
$$

Buscar $(u,p)\in \bar u+ \mathcal{V}\times \mathcal{Q}$ tal que para todo $(v,q)\in \mathcal{V}\times \mathcal{Q}$:
$$
\boxed{
\int_\Omega P(u,p):\nabla v\,\mathrm{d}x
+\int_\Omega \big(\ln J(u)-\tfrac{p}{\lambda}\big)\,q\,\mathrm{d}x
=\int_{\Gamma_N} t\cdot v\,\mathrm{d}s
}
$$
donde $P(u,p)=\partial\psi_{\mathrm{dev}}/\partial F + p\,F^{-\top}$ y $F=I+\nabla u$.

In [ ]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
from basix.ufl import element, mixed_element
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from ufl import (Identity, TestFunctions, TrialFunctions, split, 
                 grad, det, tr, inner, derivative, dx, variable, inv, ln, TrialFunction)
import ufl
import os
import tempfile
import adios4dolfinx
import pickle

# Crear malla
domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.triangle)

# Crear directorio de resultados
results_dir = tempfile.mkdtemp(prefix="fem_results_")
print(f"Results will be saved in: {results_dir}")

# ELEMENTOS ROBUSTOS
k = 2
V_el = element("Lagrange", domain.basix_cell(), k, shape=(domain.geometry.dim,))
Q_el = element("Lagrange", domain.basix_cell(), k-1)
V_mixed = mixed_element([V_el, Q_el])
V = fem.functionspace(domain, V_mixed)

# PARÁMETROS 
mu = fem.Constant(domain, 1.0)
lmbda = fem.Constant(domain, 100.0)

# Función para la solución
u_p_ = fem.Function(V)
u, p = split(u_p_)

# Funciones test
v, q = TestFunctions(V)

# FORMULACIÓN NEO-HOOKE 
I = Identity(domain.geometry.dim)
F = variable(grad(u) + I)
J = variable(det(F))
C = variable(F.T * F)
Ic = variable(tr(C))

# Definir psi_dev sin la parte volumétrica
psi_dev = (mu / 2) * (Ic - 2 - 2 * ln(J))

# Calcular P_dev = diff(psi_dev, F)
P_dev = ufl.diff(psi_dev, F)

# Añadir la parte de presión
P = P_dev + p * inv(F).T

# Forma débil corregida
G = inner(P, grad(v)) * dx + inner(ln(J) - p / lmbda, q) * dx
du = TrialFunction(V)
J_form = derivative(G, u_p_, du)

# CONDICIONES DE BORDE
def bottom(x):
    return np.isclose(x[1], 0.0)

def top(x):
    return np.isclose(x[1], 1.0)

bottom_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, bottom)
top_facets = mesh.locate_entities_boundary(domain, domain.topology.dim-1, top)

# Fija u_x = 0 y u_y = 0 en la cara inferior
V0_bottom, _ = V.sub(0).collapse()
bottom_dofs = fem.locate_dofs_topological((V.sub(0), V0_bottom), domain.topology.dim-1, bottom_facets)
u_bottom = fem.Function(V0_bottom)
u_bottom.x.array[:] = 0.0
bc_bottom = fem.dirichletbc(u_bottom, bottom_dofs, V.sub(0))

# BC superior - SOLO en componente Y
V0_y, _ = V.sub(0).sub(1).collapse()
top_dofs_y = fem.locate_dofs_topological((V.sub(0).sub(1), V0_y), domain.topology.dim-1, top_facets)

u_top_val = fem.Function(V0_y)
u_top_val.x.array[:] = 0.0
bc_top = fem.dirichletbc(u_top_val, top_dofs_y, V.sub(0).sub(1))

bcs = [bc_bottom, bc_top]
print("✓ BCs corregidas: Cara inferior fija, Cara superior desliza en X.")

# SOLVER CONFIGURADO
problem = NonlinearProblem(G, u_p_, bcs=bcs, J=J_form)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.atol = 1e-8
solver.rtol = 1e-8
solver.max_it = 50
solver.convergence_criterion = "incremental"

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "gmres"
opts[f"{option_prefix}pc_type"] = "lu"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

# CARGA ADAPTATIVA
def adaptive_load_increments(initial_disp, target_disp, max_steps=50):
    increments = []
    current = initial_disp
    
    while current < target_disp:
        increments.append(current)
        if current < target_disp * 0.1:
            step = target_disp / 30
        elif current < target_disp * 0.5:
            step = target_disp / 20
        else:
            step = target_disp / 15
            
        current += step
        if current > target_disp:
            current = target_disp
    
    increments.append(target_disp)
    return np.unique(increments)

initial_displacement = 0.01
target_displacement = 0.1
max_steps = 100

load_values = adaptive_load_increments(initial_displacement, target_displacement, max_steps)
num_steps = len(load_values) - 1

print(f"Estrategia de carga adaptativa:")
print(f"   - Desplazamiento objetivo: {target_displacement:.3f}")
print(f"   - Número de pasos: {num_steps}")

# Preparar espacios de visualización P1
V_vis = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
Q_vis = fem.functionspace(domain, ("Lagrange", 1))

print("\n=== Iniciando simulación directa ===")

convergence_history = []
all_converged = True

for i, disp in enumerate(load_values[1:]):
    step_size = disp - load_values[i]
    print(f"Paso {i+1}/{num_steps}, Desplazamiento = {disp:.4f} (Δ = {step_size:.4f})")
    
    u_top_val.x.array[:] = -disp
    
    try:
        num_its, converged = solver.solve(u_p_)
        
        if converged:
            print(f"   ✓ Converged in {num_its} iterations")
            convergence_history.append(num_its)
            
            u_sol = u_p_.sub(0).collapse()
            u_array = u_sol.x.array.reshape(-1, domain.geometry.dim)
            max_deformation_x = np.max(np.abs(u_array[:, 0]))
            max_deformation_y = np.max(np.abs(u_array[:, 1]))
            print(f"   → Deformación máxima Y: {max_deformation_y:.4f}")
            print(f"   → Deformación máxima X (abombamiento): {max_deformation_x:.4f}")
            
        else:
            print(f"   ✗ FAILED to converge after {num_its} iterations")
            all_converged = False
            break
            
    except Exception as e:
        print(f"   ✗ ERROR: {e}")
        all_converged = False
        break

print("\n=== Guardando resultados ===")

if all_converged:
    
    print("--- Guardando datos para problema inverso (Alta Fidelidad) ---")
    
    u_sol = u_p_.sub(0).collapse()
    p_sol = u_p_.sub(1).collapse()
    u_sol.name = "Displacement"
    p_sol.name = "Pressure"

    # Guardar malla
    mesh_file = os.path.join(results_dir, "simulation_mesh.bp")
    adios4dolfinx.write_mesh(mesh_file, domain, engine="BP4")

    # Guardar funciones de alta fidelidad
    disp_file = os.path.join(results_dir, "displacement.bp")
    pres_file = os.path.join(results_dir, "pressure.bp")
    
    adios4dolfinx.write_function(disp_file, u_sol, engine="BP4")
    adios4dolfinx.write_function(pres_file, p_sol, engine="BP4")
    
    print(f"Datos para inverso guardados:")
    print(f" - {os.path.basename(mesh_file)} (Malla original)")
    print(f" - {os.path.basename(disp_file)} (Desplazamiento k={k})")
    print(f" - {os.path.basename(pres_file)} (Presión k={k-1})")

    # --- 2. DATOS PARA VISUALIZACIÓN EN XDMF (P1) ---
    print("\n--- Guardando resultados en XDMF para visualización (P1) ---")
    
    # Interpolar a P1 para XDMF
    u_vis = fem.Function(V_vis)
    u_vis.name = "Displacement"
    u_vis.interpolate(u_sol)
    
    p_vis = fem.Function(Q_vis)
    p_vis.name = "Pressure"
    p_vis.interpolate(p_sol)
    
    # Guardar en XDMF (malla original)
    xdmf_file = io.XDMFFile(domain.comm, os.path.join(results_dir, "simulation_results.xdmf"), "w")
    xdmf_file.write_mesh(domain)
    xdmf_file.write_function(u_vis, 0.0)
    xdmf_file.write_function(p_vis, 0.0)
    xdmf_file.close()
    print("✓ XDMF guardado: simulation_results.xdmf")
    
    print("\n--- Creando malla deformada para visualización ---")
    
    # Calcular nuevas coordenadas
    original_coords = domain.geometry.x
    displacement_at_vertices = u_vis.x.array.reshape((-1, domain.geometry.dim))
    
    displacement_3d = np.zeros_like(original_coords)
    displacement_3d[:, :domain.geometry.dim] = displacement_at_vertices
    new_coords = original_coords + displacement_3d
    
    # Crear malla deformada
    cells_array = domain.topology.connectivity(domain.topology.dim, 0).array.reshape((-1, domain.topology.dim + 1))
    cells_array = np.asarray(cells_array, dtype=np.int64)
    deformed_mesh = mesh.create_mesh(domain.comm, cells_array, new_coords[:, :domain.geometry.dim], domain.ufl_domain())
    
    # Crear funciones en malla deformada
    V_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1, (deformed_mesh.geometry.dim,)))
    Q_vis_deformed = fem.functionspace(deformed_mesh, ("Lagrange", 1))
    
    u_vis_def = fem.Function(V_vis_deformed)
    u_vis_def.name = "Displacement"
    u_vis_def.x.array[:] = u_vis.x.array
    
    p_vis_def = fem.Function(Q_vis_deformed)
    p_vis_def.name = "Pressure"
    p_vis_def.x.array[:] = p_vis.x.array
    
    # Guardar malla deformada en XDMF
    xdmf_deformed = io.XDMFFile(deformed_mesh.comm, os.path.join(results_dir, "deformed_results.xdmf"), "w")
    xdmf_deformed.write_mesh(deformed_mesh)
    xdmf_deformed.write_function(u_vis_def, 0.0)
    xdmf_deformed.write_function(p_vis_def, 0.0)
    xdmf_deformed.close()
    print("✓ XDMF deformado guardado: deformed_results.xdmf")
    
    # También guardar con adios4dolfinx
    vis_disp_file = os.path.join(results_dir, "VISUALIZATION_displacement.bp")
    vis_pres_file = os.path.join(results_dir, "VISUALIZATION_pressure.bp")
    
    adios4dolfinx.write_function(vis_disp_file, u_vis_def, engine="BP4")
    adios4dolfinx.write_function(vis_pres_file, p_vis_def, engine="BP4")
    print("✓ Archivos BP4 guardados para ParaView")

    print(f"\n✅ Simulación completada exitosamente")
    print(f"📂 Resultados guardados en: {results_dir}")
    print(f"\n=== Instrucciones de Visualización ===")
    print("\n📊 OPCIÓN 1 - ParaView con XDMF:")
    print("  1. Abre 'simulation_results.xdmf'")
    print("  2. Aplica 'Warp By Vector' usando 'Displacement'")
    print("  3. O abre 'deformed_results.xdmf' (ya deformada)")
    print("\n📊 OPCIÓN 2 - ParaView con BP4:")
    print("  1. Abre 'VISUALIZATION_displacement.bp'")
    print("  2. La malla aparece directamente deformada")
    print("\n🔬 Para problema inverso:")
    print("  - Usa 'simulation_mesh.bp'")
    print("  - Usa 'displacement.bp' (k=2)")
    print("  - Usa 'pressure.bp' (k=1)")

else:
    print(f"✗ Simulación no convergente. No se guardaron resultados en: {results_dir}")

Results will be saved in: /tmp/fem_results_vgj2c4qr
✓ BCs corregidas: Cara inferior fija, Cara superior desliza en X.
Estrategia de carga adaptativa:
   - Desplazamiento objetivo: 0.100
   - Número de pasos: 17

=== Iniciando simulación directa ===
Paso 1/17, Desplazamiento = 0.0133 (Δ = 0.0033)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0133
   → Deformación máxima X (abombamiento): 0.0076
Paso 2/17, Desplazamiento = 0.0183 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0183
   → Deformación máxima X (abombamiento): 0.0105
Paso 3/17, Desplazamiento = 0.0233 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0233
   → Deformación máxima X (abombamiento): 0.0135
Paso 4/17, Desplazamiento = 0.0283 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima Y: 0.0283
   → Deformación máxima X (abombamiento): 0.0164
Paso 5/17, Desplazamiento = 0.0333 (Δ = 0.0050)
   ✓ Converged in 4 iterations
   → Deformación máxima 

# Formulación Mixta Desplazamiento–Presión en Elasticidad Casi Incomprensible

## 1. Espacios de Funciones Discretas

Se considera un problema de elasticidad casi incomprensible formulado en términos mixtos desplazamiento–presión. El espacio de funciones discretas se define como un producto cartesiano de dos subespacios finitos:

$$V_h = V_{u,h} \times V_{p,h}$$

### 1.1 Espacio de Desplazamientos

El espacio de desplazamientos se define como:

$$V_{u,h} = \{\, u_h : \Omega \to \mathbb{R}^d \mid u_h \text{ es vectorial por componentes en } \mathbb{P}_2(K) \ \forall K \in \mathcal{T}_h \,\}$$

### 1.2 Espacio de Presiones

El espacio de presiones se define como:

$$V_{p,h} = \{\, p_h : \Omega \to \mathbb{R} \mid p_h \in \mathbb{P}_1(K) \ \forall K \in \mathcal{T}_h \,\}$$

### 1.3 Notación

- $\mathbb{P}_k(K)$: espacio de polinomios de grado menor o igual a $k$ en el elemento $K$
- $\mathcal{T}_h$: partición (triangular en 2D) del dominio $\Omega$
- $d$: dimensión espacial (en el código, $d=2$)

### 1.4 Espacio Mixto de Taylor–Hood

El espacio mixto utilizado es el par de elementos de Taylor–Hood $\mathbb{P}_2$–$\mathbb{P}_1$:

$$V_h = [\mathbb{P}_2]^d \times \mathbb{P}_1$$

---

## 2. Formulación Variacional Mixta

### 2.1 Pequeñas Deformaciones

En el régimen de pequeñas deformaciones, la formulación abstracta del problema mixto es: encontrar $(u_h, p_h) \in V_{u,h} \times V_{p,h}$ tal que

$$\begin{cases}
a(u_h, v_h) + b(v_h, p_h) = f(v_h) & \forall v_h \in V_{u,h} \\
b(u_h, q_h) - \dfrac{1}{\lambda}\,(p_h, q_h) = 0 & \forall q_h \in V_{p,h}
\end{cases}$$

### 2.2 Formas Bilineales

**Forma bilineal elástica:**

$a(\cdot,\cdot)$ es la forma bilineal elástica asociada a la parte desviadora (dependiente de $\mu$).

**Forma de acoplamiento volumen–presión:**

$$b(v,q) = \int_\Omega q\, \operatorname{div} v \, dx$$

**Producto interno:**

$(\cdot,\cdot)$ denota el producto interno $L^2(\Omega)$.

### 2.3 Parámetros

- $\lambda$: parámetro de Lamé volumétrico (grande en el régimen casi incomprensible)
- $f$: funcional de cargas

---

## 3. Grandes Deformaciones

En el caso de grandes deformaciones, la estructura es análoga, pero la restricción de incomprensibilidad se formula en términos de $J = \det F$, donde $F = I + \nabla u$ es el gradiente de deformación.

El término de acoplamiento en la forma débil es:

$$\int_\Omega (\ln J(u_h) - p_h/\lambda)\, q_h \, dx$$

Este término juega el mismo papel que $b(u_h,q_h)$ en la formulación lineal, enlazando el campo de desplazamientos $u_h$ con la presión $p_h$.

---

## 4. Condición de Estabilidad: Condición Inf–Sup

La condición clave para la estabilidad y buena formulación del problema mixto es la **condición inf–sup** (también llamada condición de Ladyzhenskaya–Babuška–Brezzi).

### 4.1 Enunciado de la Condición

Dada la forma bilineal de acoplamiento

$$b : V_{u,h} \times V_{p,h} \to \mathbb{R}, \qquad b(v_h, q_h) \approx \int_\Omega q_h\, \operatorname{div} v_h \, dx$$

se exige que exista una constante $\beta > 0$, independiente del tamaño de malla $h$, tal que

$$\inf_{0 \ne q_h \in V_{p,h}}
\sup_{0 \ne v_h \in V_{u,h}}
\frac{b(v_h, q_h)}{\|v_h\|_{1} \, \|q_h\|_{0}}
\;\ge\; \beta$$

### 4.2 Implicaciones de la Condición Inf–Sup

Esta desigualdad garantiza que:

1. **Ausencia de modos espurios de presión:** No existan modos de presión discretos $q_h$ tales que $b(v_h,q_h)=0$ para todo $v_h\in V_{u,h}$.

2. **Buen condicionamiento:** El problema discretizado sea bien condicionado al refinar la malla y que la presión esté controlada en función de los desplazamientos.

3. **Estabilidad en el límite casi incomprensible:** La solución $(u_h,p_h)$ exista, sea única (bajo las hipótesis habituales) y se comporte de manera estable en el límite casi incomprensible ($\lambda \to \infty$) sin sufrir locking volumétrico severo.

---

## 5. Validez del Par P₂–P₁ (Taylor–Hood)

El par $\mathbb{P}_2$–$\mathbb{P}_1$ (Taylor–Hood) es un ejemplo clásico de espacios $(V_{u,h}, V_{p,h})$ que satisfacen la condición inf–sup sobre mallas triangulares o tetraédricas regulares.

### 5.1 Espacios

$$V_{u,h} = [\mathbb{P}_2]^d, \qquad V_{p,h} = \mathbb{P}_1$$

### 5.2 Propiedades

Estos espacios constituyen un par inf–sup estable para:

- Problemas tipo Stokes
- Formulaciones mixtas de elasticidad incomprensible
- Formulaciones mixtas de elasticidad casi incomprensible

La elección de espacios con orden polinómico mayor para $u_h$ que para $p_h$ asegura que la presión discreta no tenga "grados de libertad de más" respecto de los desplazamientos, lo que sería una fuente de inestabilidad.

---

## 6. Conclusión

Al tomar

$$V_h = V_{u,h} \times V_{p,h} = [\mathbb{P}_2]^d \times \mathbb{P}_1$$

se obtiene un espacio mixto que:

- ✓ Respeta la condición inf–sup discreta
- ✓ Produce campos de desplazamiento y presión numéricamente estables
- ✓ Es adecuado para modelar materiales casi incomprensibles sin bloqueo volumétrico
- ✓ Funciona tanto en la resolución del problema directo como en el análisis inverso asociado

# Locking Volumétrico en Elasticidad Nearly Incompresible

## 1. Modelo Continuo: Elasticidad Casi Incomprensible

### 1.1 Formulación del Problema

Considérese un cuerpo elástico en un dominio $\Omega \subset \mathbb{R}^d$ (con $d = 2, 3$), con desplazamiento $u : \Omega \to \mathbb{R}^d$, sometido a fuerzas de cuerpo $f$ y condiciones de borde adecuadas.

### 1.2 Tensor de Tensiones de Cauchy

En elasticidad lineal isotrópica, el tensor de tensiones de Cauchy viene dado por

$$\sigma(u) = 2\mu\,\varepsilon(u) + \lambda\,\operatorname{tr}(\varepsilon(u))\,I$$

donde:

- $\varepsilon(u) = \tfrac12(\nabla u + \nabla u^{T})$ es el tensor de deformación lineal
- $\mu > 0$ es el parámetro de Lamé de cortante
- $\lambda \ge 0$ es el parámetro de Lamé volumétrico
- $\operatorname{tr}(\varepsilon)$ describe la deformación volumétrica (cambio de volumen local)

### 1.3 Ecuaciones de Equilibrio (Forma Fuerte)

$$\begin{cases}
 -\nabla\cdot \sigma(u) = f & \text{en } \Omega \\
 u = 0 & \text{en } \Gamma_D \\
 \sigma(u)n = t & \text{en } \Gamma_N
\end{cases}$$

con descomposición $\partial\Omega = \Gamma_D \cup \Gamma_N$.

### 1.4 Formulación Variacional

Encontrar $u \in V := \{ v \in [H^1(\Omega)]^d : v = 0 \text{ en } \Gamma_D \}$ tal que

$$a(u,v) = \ell(v) \quad \forall v \in V$$

donde

$$a(u,v) := \int_\Omega \Big( 2\mu\,\varepsilon(u):\varepsilon(v) + \lambda\,\operatorname{tr}(\varepsilon(u))\,\operatorname{tr}(\varepsilon(v)) \Big)\,dx$$

$$\ell(v) := \int_\Omega f\cdot v\,dx + \int_{\Gamma_N} t\cdot v\,ds$$

### 1.5 Régimen Nearly Incompresible

El régimen casi incomprensible corresponde a $\nu \to 0.5$, lo que equivale a $\lambda \to +\infty$.

En el límite $\lambda \to \infty$, la solución exacta $u$ debe satisfacer aproximadamente la restricción

$$\operatorname{tr}(\varepsilon(u)) \approx 0 \quad \text{en } \Omega$$

es decir, el material apenas puede cambiar de volumen.

---

## 2. Aproximación por Elementos Finitos y Dependencia en $\lambda$

### 2.1 Problema Discreto Primal

Sea $\mathcal{T}_h$ una partición de $\Omega$ y sea $V_h \subset V$ un subespacio finito (por ejemplo, funciones por partes polinomiales continuas de cierto orden).

El problema discreto puramente primal consiste en: encontrar $u_h \in V_h$ tal que

$$a(u_h,v_h) = \ell(v_h) \quad \forall v_h \in V_h$$

### 2.2 Soluciones Exacta y Discreta

Se define:

- Solución exacta $u^\lambda$: solución del problema continuo para un valor dado de $\lambda$
- Solución discreta $u_h^\lambda$: solución FEM correspondiente

### 2.3 Convergencia Óptima y Robusta

Un esquema FEM es óptimamente convergente y robusto si existen constantes $C_1, C_2 > 0$, independientes de $h$ y de $\lambda$, tales que

$$\|u^\lambda - u_h^\lambda\|_{1} \le C_1 \inf_{v_h \in V_h} \|u^\lambda - v_h\|_{1}$$

$$\|u^\lambda\|_{1} \le C_2 \|\ell\|_{V'}$$

es decir, si la constante de estabilidad y las cotas de error no se degradan al hacer $\lambda \to \infty$.

### 2.4 Pérdida de Estabilidad Uniforme

En la práctica, cuando se utiliza un espacio $V_h$ únicamente de desplazamientos estándar (por ejemplo, elementos lineales $\mathbb{P}_1$), estas constantes sí dependen fuertemente de $\lambda$, y típicamente crecen sin cota al aumentar $\lambda$.

Formalmente, existe una constante $C(\lambda)$ tal que

$$\|u^\lambda - u_h^\lambda\|_1 \le C(\lambda)\, \inf_{v_h \in V_h} \|u^\lambda - v_h\|_1$$

y se observa que

$$C(\lambda) \to +\infty \quad \text{cuando } \lambda \to +\infty$$

**Esta pérdida de estabilidad uniforme es el núcleo del fenómeno de locking.**

---

## 3. Definición Formal de Locking Volumétrico

Se dice que un método de elementos finitos sufre **locking volumétrico** si, en el régimen casi incomprensible ($\lambda$ grande), se cumple alguna de las siguientes situaciones (equivalentes desde el punto de vista numérico):

### 3.1 No Uniformidad del Error en el Límite Incomprensible

Para un $h$ fijo, la solución discreta $u_h^\lambda$ no converge a la solución límite incomprensible $u^\infty$ (solución del problema con restricción $\operatorname{tr}(\varepsilon(u)) = 0$), sino que

$$\lim_{\lambda \to \infty} \|u_h^\lambda - u^\infty\|_1 \not\to 0$$

e incluso puede suceder que

$$\lim_{\lambda \to \infty} u_h^\lambda = 0$$

en un sentido adecuado, aun cuando $u^\infty$ sea no trivial.

### 3.2 Rigidez Numérica Artificial

La energía elástica discreta

$$E_h^\lambda = \frac12\, a(u_h^\lambda, u_h^\lambda)$$

crece de manera no física con $\lambda$ y domina por términos volumétricos mal representados, haciendo que las deformaciones discretas sean mucho más pequeñas que las que predice el modelo continuo.

### 3.3 Degradación de la Tasa de Convergencia

A medida que $\lambda$ aumenta, la tasa de convergencia esperada (por ejemplo, $O(h^k)$ para elementos de orden $k$) se pierde, y los errores dejan de disminuir de manera acorde al refinamiento de malla.

En muchos casos, existe un umbral a partir del cual refinar la malla no mejora significativamente la solución, debido a la rigidez volumétrica numérica.

---

## 4. Mecanismo Físico del Locking

### 4.1 Sobre-restricción de la Deformación

En resumen, locking volumétrico describe el comportamiento de un esquema FEM en el que la imposición (explícita o implícita) de incompresibilidad a través de $\lambda$ grandes conduce a una sobre–restricción de la deformación volumétrica en el espacio discreto $V_h$.

### 4.2 Carencias del Espacio Discreto

De tal modo que:

- El espacio $V_h$ no contiene suficientes funciones "casi incomprensibles" que aproximen bien al $u^\lambda$ físico
- El esquema pierde estabilidad y capacidad de aproximación uniforme en el límite $\lambda \to \infty$
- La respuesta estructural se vuelve artificialmente rígida por un efecto puramente numérico, no físico

### 4.3 Origen del Término "Volumétrico"

Este fenómeno se denomina **volumétrico** porque está directamente asociado al:

- Término volumétrico $\lambda\,\operatorname{tr}(\varepsilon(u))$ en formulaciones de pequeñas deformaciones
- Penalización del determinante $J = \det F$ en formulaciones de grandes deformaciones

---

## 5. Solución: Formulaciones Mixtas

Un método que evita el locking volumétrico es aquel que conserva **estabilidad y precisión uniformes en el límite** $\lambda \to \infty$, típicamente mediante:

### 5.1 Aproximación Mixta

Una **formulación mixta** (por ejemplo, desplazamiento–presión) que introduce:

- Un espacio de presiones $V_{p,h}$ adecuado
- Una condición inf–sup uniforme

### 5.2 Beneficios

Esta aproximación permite que el esquema FEM mantenga robustez y convergencia uniforme incluso cuando los parámetros del material conducen al régimen nearly incompresible.

In [2]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
from basix.ufl import element, mixed_element
from dolfinx import fem, io, mesh
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from ufl import (Identity, TestFunctions, TrialFunctions, split, 
                 grad, det, tr, inner, derivative, dx, variable, inv, ln, TrialFunction)
import ufl
import os
import tempfile
import adios4dolfinx

# CONFIGURACIÓN DEL PROBLEMA INVERSO
print("="*70)
print("=== PROBLEMA INVERSO: RECUPERAR CONFIGURACIÓN ORIGINAL ===")
print("="*70)

# Directorio donde están los resultados del problema directo
direct_results_dir = "/tmp/fem_results_57g4p8ye"  # CAMBIAR por tu directorio
print(f"\n📁 Buscando resultados del problema directo en:")
print(f"   {direct_results_dir}")

# CARGAR MALLA Y SOLUCIÓN DEL PROBLEMA DIRECTO
print("\n--- Cargando datos del problema directo ---")

# Leer la malla original
mesh_file = os.path.join(direct_results_dir, "simulation_mesh.bp")
mesh_reference = adios4dolfinx.read_mesh(mesh_file, engine="BP4", comm=MPI.COMM_WORLD)
print(f"✓ Malla de referencia cargada")
print(f"  - Dimensiones: {mesh_reference.geometry.x.shape}")
print(f"  - Rango Y original: [{mesh_reference.geometry.x[:, 1].min():.3f}, {mesh_reference.geometry.x[:, 1].max():.3f}]")

# Leer el desplazamiento del problema directo
disp_file = os.path.join(direct_results_dir, "displacement.bp")

# Necesitamos crear el espacio de funciones antes de leer
k = 2  # Mismo orden que el problema directo
V_direct = fem.functionspace(mesh_reference, ("Lagrange", k, (mesh_reference.geometry.dim,)))
u_direct = fem.Function(V_direct)
u_direct.name = "Displacement"  # Nombre correcto

# Leer la función (modifica u_direct in-place, no devuelve nada útil)
adios4dolfinx.read_function(disp_file, u_direct, time=0.0, engine="BP4")
print(f"✓ Desplazamiento del problema directo cargado (k={k})")
print(f"  - Rango de desplazamiento: [{u_direct.x.array.min():.6f}, {u_direct.x.array.max():.6f}]")

#  CREAR MALLA DEFORMADA (CONFIGURACIÓN ESPACIAL)
print("\n--- Creando configuración espacial (deformada) ---")

# La configuración espacial es: x_spatial = x_reference + u_direct
original_coords = mesh_reference.geometry.x.copy()

# Interpolar el desplazamiento k=2 a P1 para obtener valores en los vértices
V_P1 = fem.functionspace(mesh_reference, ("Lagrange", 1, (mesh_reference.geometry.dim,)))
u_direct_P1 = fem.Function(V_P1)
u_direct_P1.interpolate(u_direct)

# Ahora u_direct_P1 tiene exactamente un DOF por vértice por componente
u_direct_array = u_direct_P1.x.array.reshape(-1, mesh_reference.geometry.dim)
print(f"  - DOFs del desplazamiento P1: {u_direct_P1.x.array.shape}")
print(f"  - Vértices de la malla: {original_coords.shape[0]}")

# Crear nueva malla deformada
new_coords_3d = original_coords.copy()
new_coords_3d[:, :mesh_reference.geometry.dim] += u_direct_array

cells_array = mesh_reference.topology.connectivity(mesh_reference.topology.dim, 0).array.reshape(
    (-1, mesh_reference.topology.dim + 1))
cells_array = np.asarray(cells_array, dtype=np.int64)

mesh_spatial = mesh.create_mesh(
    mesh_reference.comm, 
    cells_array, 
    new_coords_3d[:, :mesh_reference.geometry.dim], 
    mesh_reference.ufl_domain()
)

print(f"✓ Malla deformada (configuración espacial) creada")
print(f"  - Rango Y deformado: [{mesh_spatial.geometry.x[:, 1].min():.3f}, {mesh_spatial.geometry.x[:, 1].max():.3f}]")

# Calcular la compresión real
original_height = original_coords[:, 1].max() - original_coords[:, 1].min()
deformed_height = mesh_spatial.geometry.x[:, 1].max() - mesh_spatial.geometry.x[:, 1].min()
compression_ratio = deformed_height / original_height
print(f"  - Compresión: {compression_ratio:.2%} de la altura original")
print(f"  - Altura original: {original_height:.4f}")
print(f"  - Altura deformada: {deformed_height:.4f}")

# Crear directorio para resultados inversos
results_dir_inverse = tempfile.mkdtemp(prefix="fem_inverse_results_")
print(f"\n📂 Resultados inversos se guardarán en: {results_dir_inverse}")

#  ELEMENTOS PARA PROBLEMA INVERSO 
print("\n--- Configurando problema inverso ---")

V_el = element("Lagrange", mesh_spatial.basix_cell(), k, shape=(mesh_spatial.geometry.dim,))
Q_el = element("Lagrange", mesh_spatial.basix_cell(), k-1)
V_mixed = mixed_element([V_el, Q_el])
V = fem.functionspace(mesh_spatial, V_mixed)

#  PARÁMETROS DEL MATERIAL 
mu = fem.Constant(mesh_spatial, 1.0)
lmbda = fem.Constant(mesh_spatial, 100.0)

# Función para la solución inversa
u_p_inverse = fem.Function(V)
u_inverse, p_inverse = split(u_p_inverse)

# Funciones test
v, q = TestFunctions(V)

#  FORMULACIÓN NEO-HOOKE PARA PROBLEMA INVERSO 
I = Identity(mesh_spatial.geometry.dim)

# F_inverse mapea de configuración ESPACIAL (deformada) a MATERIAL (original)
F_inverse = variable(grad(u_inverse) + I)
J_inverse = variable(det(F_inverse))
C_inverse = variable(F_inverse.T * F_inverse)
Ic_inverse = variable(tr(C_inverse))

# Energía (misma forma funcional que el problema directo)
psi_dev = (mu / 2) * (Ic_inverse - 2 - 2 * ln(J_inverse))
P_dev = ufl.diff(psi_dev, F_inverse)
P_inverse = P_dev + p_inverse * inv(F_inverse).T

# Forma débil
G_inverse = inner(P_inverse, grad(v)) * dx + inner(ln(J_inverse) - p_inverse / lmbda, q) * dx
du_inverse = TrialFunction(V)
J_form_inverse = derivative(G_inverse, u_p_inverse, du_inverse)

print("✓ Formulación Neo-Hooke configurada")

#  CONDICIONES DE BORDE 
print("\n--- Configurando condiciones de borde ---")

# En la configuración espacial (deformada)
def bottom_spatial(x):
    y_min = mesh_spatial.geometry.x[:, 1].min()
    return np.isclose(x[1], y_min, atol=1e-3)

def top_spatial(x):
    y_max = mesh_spatial.geometry.x[:, 1].max()
    return np.isclose(x[1], y_max, atol=1e-3)

bottom_facets = mesh.locate_entities_boundary(mesh_spatial, mesh_spatial.topology.dim-1, bottom_spatial)
top_facets = mesh.locate_entities_boundary(mesh_spatial, mesh_spatial.topology.dim-1, top_spatial)

print(f"✓ Facetas localizadas:")
print(f"  - Inferiores: {len(bottom_facets)}")
print(f"  - Superiores: {len(top_facets)}")

# BC inferior: fijar completamente (como en el problema directo)
V0_bottom, _ = V.sub(0).collapse()
bottom_dofs = fem.locate_dofs_topological((V.sub(0), V0_bottom), mesh_spatial.topology.dim-1, bottom_facets)
u_bottom = fem.Function(V0_bottom)
u_bottom.x.array[:] = 0.0
bc_bottom = fem.dirichletbc(u_bottom, bottom_dofs, V.sub(0))

# BC superior: SOLO componente Y (permitir deslizamiento en X)
V0_y, _ = V.sub(0).sub(1).collapse()
top_dofs_y = fem.locate_dofs_topological((V.sub(0).sub(1), V0_y), mesh_spatial.topology.dim-1, top_facets)
u_top_val = fem.Function(V0_y)
u_top_val.x.array[:] = 0.0  # Se actualizará en el loop
bc_top = fem.dirichletbc(u_top_val, top_dofs_y, V.sub(0).sub(1))

bcs = [bc_bottom, bc_top]
print("✓ BCs configuradas: Base fija, tope desliza en X")

# ========== SOLVER PARA PROBLEMA INVERSO ==========
problem_inverse = NonlinearProblem(G_inverse, u_p_inverse, bcs=bcs, J=J_form_inverse)
solver_inverse = NewtonSolver(MPI.COMM_WORLD, problem_inverse)

solver_inverse.atol = 1e-8
solver_inverse.rtol = 1e-8
solver_inverse.max_it = 50
solver_inverse.convergence_criterion = "incremental"

ksp_inverse = solver_inverse.krylov_solver
opts_inverse = PETSc.Options()
option_prefix_inverse = ksp_inverse.getOptionsPrefix()
opts_inverse[f"{option_prefix_inverse}ksp_type"] = "gmres"
opts_inverse[f"{option_prefix_inverse}pc_type"] = "lu"
opts_inverse[f"{option_prefix_inverse}pc_factor_mat_solver_type"] = "mumps"
ksp_inverse.setFromOptions()

print("✓ Solver configurado")

#  ESTRATEGIA DE CARGA INCREMENTAL 
print("\n--- Configurando estrategia de carga ---")

# Necesitamos llevar la configuración deformada de vuelta a la original
# El desplazamiento material debe ser POSITIVO en Y (expandir)
target_expansion = original_height - deformed_height

# Estrategia incremental
def adaptive_expansion_increments(initial_disp, target_disp, max_steps=30):
    increments = []
    current = initial_disp
    
    while current < target_disp:
        increments.append(current)
        if current < target_disp * 0.2:
            step = target_disp / 30
        elif current < target_disp * 0.6:
            step = target_disp / 20
        else:
            step = target_disp / 15
            
        current += step
        if current > target_disp:
            current = target_disp
    
    increments.append(target_disp)
    return np.unique(increments)

initial_displacement = 0.005
load_values = adaptive_expansion_increments(initial_displacement, target_expansion, max_steps=30)
num_steps = len(load_values) - 1

print(f"  - Expansión objetivo: {target_expansion:.4f}")
print(f"  - Número de pasos: {num_steps}")

#  RESOLVER PROBLEMA INVERSO 
print("\n" + "="*70)
print("=== INICIANDO SOLUCIÓN DEL PROBLEMA INVERSO ===")
print("="*70)

convergence_history = []
all_converged = True

for i, expansion in enumerate(load_values[1:]):
    step_size = expansion - load_values[i]
    print(f"\nPaso {i+1}/{num_steps}, Expansión = {expansion:.4f} (Δ = {step_size:.4f})")
    
    # Actualizar BC superior (desplazamiento POSITIVO en Y)
    u_top_val.x.array[:] = expansion
    
    try:
        num_its, converged = solver_inverse.solve(u_p_inverse)
        
        if converged:
            print(f"   ✓ Converged in {num_its} iterations")
            convergence_history.append(num_its)
            
            u_sol = u_p_inverse.sub(0).collapse()
            u_array = u_sol.x.array.reshape(-1, mesh_spatial.geometry.dim)
            max_expansion_x = np.max(np.abs(u_array[:, 0]))
            max_expansion_y = np.max(np.abs(u_array[:, 1]))
            print(f"   → Desplazamiento máximo Y: {max_expansion_y:.4f}")
            print(f"   → Desplazamiento máximo X: {max_expansion_x:.4f}")
            
        else:
            print(f"   ✗ FAILED to converge after {num_its} iterations")
            all_converged = False
            break
            
    except Exception as e:
        print(f"   ✗ ERROR: {e}")
        all_converged = False
        break

print("\n" + "="*70)
print("=== GUARDANDO RESULTADOS ===")
print("="*70)

if all_converged:
    
    # SOLUCIÓN 
    print("\n--- Guardando solución de alta fidelidad ---")
    
    u_sol = u_p_inverse.sub(0).collapse()
    p_sol = u_p_inverse.sub(1).collapse()
    u_sol.name = "Material_Displacement"
    p_sol.name = "Material_Pressure"
    
    # Guardar malla espacial
    mesh_spatial_file = os.path.join(results_dir_inverse, "spatial_mesh.bp")
    adios4dolfinx.write_mesh(mesh_spatial_file, mesh_spatial, engine="BP4")
    
    # Guardar solución
    disp_inv_file = os.path.join(results_dir_inverse, "material_displacement.bp")
    pres_inv_file = os.path.join(results_dir_inverse, "material_pressure.bp")
    
    adios4dolfinx.write_function(disp_inv_file, u_sol, engine="BP4")
    adios4dolfinx.write_function(pres_inv_file, p_sol, engine="BP4")
    
    print(f"✓ Datos de alta fidelidad guardados")
    
    # --- 2. CREAR CONFIGURACIÓN MATERIAL RECONSTRUIDA ---
    print("\n--- Creando configuración material reconstruida ---")
    
    # Interpolar a P1
    V_vis = fem.functionspace(mesh_spatial, ("Lagrange", 1, (mesh_spatial.geometry.dim,)))
    Q_vis = fem.functionspace(mesh_spatial, ("Lagrange", 1))
    
    u_vis = fem.Function(V_vis)
    u_vis.name = "Material_Displacement"
    u_vis.interpolate(u_sol)
    
    p_vis = fem.Function(Q_vis)
    p_vis.name = "Material_Pressure"
    p_vis.interpolate(p_sol)
    
    # Calcular coordenadas materiales: X = x_spatial + u_inverse
    displacement_field = u_vis.x.array.reshape(-1, mesh_spatial.geometry.dim)
    material_coords_3d = mesh_spatial.geometry.x.copy()
    material_coords_3d[:, :mesh_spatial.geometry.dim] += displacement_field
    
    mesh_material = mesh.create_mesh(
        mesh_spatial.comm,
        cells_array,
        material_coords_3d[:, :mesh_spatial.geometry.dim],
        mesh_spatial.ufl_domain()
    )
    
    print(f"✓ Configuración material reconstruida")
    print(f"  - Rango Y: [{mesh_material.geometry.x[:, 1].min():.3f}, {mesh_material.geometry.x[:, 1].max():.3f}]")
    
    # --- 3. GUARDAR ARCHIVOS XDMF PARA PARAVIEW ---
    print("\n--- Guardando archivos XDMF ---")
    
    # Archivo 1: Configuración espacial con campo de desplazamiento
    xdmf_spatial = io.XDMFFile(mesh_spatial.comm, 
                               os.path.join(results_dir_inverse, "spatial_with_displacement.xdmf"), "w")
    xdmf_spatial.write_mesh(mesh_spatial)
    xdmf_spatial.write_function(u_vis, 0.0)
    xdmf_spatial.write_function(p_vis, 0.0)
    xdmf_spatial.close()
    print("✓ spatial_with_displacement.xdmf")
    
    # Archivo 2: Configuración material reconstruida
    V_mat = fem.functionspace(mesh_material, ("Lagrange", 1, (mesh_material.geometry.dim,)))
    Q_mat = fem.functionspace(mesh_material, ("Lagrange", 1))
    
    u_mat = fem.Function(V_mat)
    u_mat.name = "Material_Displacement"
    u_mat.x.array[:] = u_vis.x.array
    
    p_mat = fem.Function(Q_mat)
    p_mat.name = "Material_Pressure"
    p_mat.x.array[:] = p_vis.x.array
    
    xdmf_material = io.XDMFFile(mesh_material.comm,
                                os.path.join(results_dir_inverse, "material_reconstructed.xdmf"), "w")
    xdmf_material.write_mesh(mesh_material)
    xdmf_material.write_function(u_mat, 0.0)
    xdmf_material.write_function(p_mat, 0.0)
    xdmf_material.close()
    print("✓ material_reconstructed.xdmf")
    
    # Archivo 3: Configuración de referencia original
    xdmf_reference = io.XDMFFile(mesh_reference.comm,
                                 os.path.join(results_dir_inverse, "reference_original.xdmf"), "w")
    xdmf_reference.write_mesh(mesh_reference)
    xdmf_reference.close()
    print("✓ reference_original.xdmf")
    
    # ANÁLISIS DE RESULTADOS 
    print("\n" + "="*70)
    print("=== ANÁLISIS DE RECONSTRUCCIÓN ===")
    print("="*70)
    
    ref_height = original_coords[:, 1].max() - original_coords[:, 1].min()
    spatial_height = mesh_spatial.geometry.x[:, 1].max() - mesh_spatial.geometry.x[:, 1].min()
    material_height = mesh_material.geometry.x[:, 1].max() - mesh_material.geometry.x[:, 1].min()
    
    print(f"\n1. CONFIGURACIÓN DE REFERENCIA (original):")
    print(f"   - Altura: {ref_height:.6f}")
    print(f"   - Rango Y: [{original_coords[:, 1].min():.3f}, {original_coords[:, 1].max():.3f}]")
    
    print(f"\n2. CONFIGURACIÓN ESPACIAL (deformada - punto de partida):")
    print(f"   - Altura: {spatial_height:.6f}")
    print(f"   - Rango Y: [{mesh_spatial.geometry.x[:, 1].min():.3f}, {mesh_spatial.geometry.x[:, 1].max():.3f}]")
    print(f"   - Compresión: {(1 - spatial_height/ref_height)*100:.1f}%")
    
    print(f"\n3. CONFIGURACIÓN MATERIAL (reconstruida - punto de llegada):")
    print(f"   - Altura: {material_height:.6f}")
    print(f"   - Rango Y: [{mesh_material.geometry.x[:, 1].min():.3f}, {mesh_material.geometry.x[:, 1].max():.3f}]")
    print(f"   - Expansión: {(material_height/spatial_height - 1)*100:.1f}%")
    
    # Error de reconstrucción
    error_height = abs(ref_height - material_height)
    error_percent = (error_height / ref_height) * 100
    
    print(f"\n" + "="*70)
    print("=== EVALUACIÓN DE LA RECONSTRUCCIÓN ===")
    print("="*70)
    print(f"Error absoluto en altura: {error_height:.6f}")
    print(f"Error relativo:           {error_percent:.2f}%")
    
    if error_percent < 1.0:
        print("\n🎯 ¡RECONSTRUCCIÓN EXCELENTE! Error < 1%")
    elif error_percent < 5.0:
        print("\n✓ Reconstrucción buena. Error < 5%")
    else:
        print("\n⚠️  Reconstrucción con error significativo. Verificar parámetros.")
    
    # Análisis del campo de desplazamiento
    u_array = u_vis.x.array.reshape(-1, mesh_spatial.geometry.dim)
    print(f"\nCampo de desplazamiento material:")
    print(f"  - Máximo en X: {np.max(np.abs(u_array[:, 0])):.6f}")
    print(f"  - Máximo en Y: {np.max(np.abs(u_array[:, 1])):.6f}")
    print(f"  - Magnitud media: {np.mean(np.linalg.norm(u_array, axis=1)):.6f}")
    
    # 
    print("\n" + "="*70)
    print("=== INSTRUCCIONES PARA PARAVIEW ===")
    print("="*70)
    print(f"\n📂 Directorio: {results_dir_inverse}\n")
    
    print("OPCIÓN 1 - Visualizar el proceso inverso completo:")
    print("  1. Abrir: spatial_with_displacement.xdmf")
    print("  2. Aplicar filtro: 'Warp By Vector'")
    print("  3. Seleccionar 'Material_Displacement' como vector")
    print("  4. Ajustar 'Scale Factor' = 1.0")
    print("  5. ¡Ver cómo la malla comprimida se expande a la original!")
    print()
    
    print("OPCIÓN 2 - Comparar las tres configuraciones:")
    print("  1. Abrir: reference_original.xdmf (configuración original)")
    print("  2. Abrir: spatial_with_displacement.xdmf (configuración deformada)")
    print("  3. Abrir: material_reconstructed.xdmf (configuración reconstruida)")
    print("  4. Comparar visualmente las tres mallas")
    print()
    
    print("OPCIÓN 3 - Analizar el campo de desplazamiento:")
    print("  1. Abrir: spatial_with_displacement.xdmf")
    print("  2. Colorear por: 'Material_Displacement' (magnitud)")
    print("  3. Ver la distribución del desplazamiento necesario para recuperar")
    print("     la configuración original")
    print()
    
    print("="*70)
    print("✅ PROBLEMA INVERSO COMPLETADO EXITOSAMENTE")
    print("="*70)
    print(f"\n📊 Convergencia promedio: {np.mean(convergence_history):.1f} iteraciones")
    print(f"📂 Resultados guardados en: {results_dir_inverse}")

else:
    print(f"\n✗ Problema inverso no convergió.")
    print(f"📂 Resultados parciales en: {results_dir_inverse}")

=== PROBLEMA INVERSO: RECUPERAR CONFIGURACIÓN ORIGINAL ===

📁 Buscando resultados del problema directo en:
   /tmp/fem_results_57g4p8ye

--- Cargando datos del problema directo ---
✓ Malla de referencia cargada
  - Dimensiones: (1089, 3)
  - Rango Y original: [0.000, 1.000]
✓ Desplazamiento del problema directo cargado (k=2)
  - Rango de desplazamiento: [-0.100000, 0.061507]

--- Creando configuración espacial (deformada) ---
  - DOFs del desplazamiento P1: (2178,)
  - Vértices de la malla: 1089
✓ Malla deformada (configuración espacial) creada
  - Rango Y deformado: [-0.000, 0.900]
  - Compresión: 90.00% de la altura original
  - Altura original: 1.0000
  - Altura deformada: 0.9000

📂 Resultados inversos se guardarán en: /tmp/fem_inverse_results_an2g7_jh

--- Configurando problema inverso ---
✓ Formulación Neo-Hooke configurada

--- Configurando condiciones de borde ---
✓ Facetas localizadas:
  - Inferiores: 32
  - Superiores: 32
✓ BCs configuradas: Base fija, tope desliza en X
✓ Solv

# Formulación Matemática Rigurosa del Problema Inverso

## 1. Configuraciones y Mapeos

Sea $d \in \{2, 3\}$.

### 1.1 Configuración de Referencia (Directa)

Se parte de un cuerpo elástico con configuración material (de referencia):

$$\Omega_{\mathrm{ref}} \subset \mathbb{R}^d, \quad X \in \Omega_{\mathrm{ref}}$$

En el problema directo se considera un mapeo de deformación:

$$\varphi_{\mathrm{dir}} : \Omega_{\mathrm{ref}} \to \mathbb{R}^d, \quad x = \varphi_{\mathrm{dir}}(X)$$

que se suele escribir como:

$$\varphi_{\mathrm{dir}}(X) = X + u_{\mathrm{dir}}(X)$$

donde $u_{\mathrm{dir}} : \Omega_{\mathrm{ref}} \to \mathbb{R}^d$ es el campo de desplazamientos.

La imagen:

$$\Omega_{\mathrm{sp}} := \varphi_{\mathrm{dir}}(\Omega_{\mathrm{ref}}) \subset \mathbb{R}^d$$

es la **configuración espacial deformada** (por ejemplo, comprimida).

### 1.2 Configuración Espacial (Inversa)

En el problema inverso se toma como dominio de cálculo precisamente:

$$\Omega := \Omega_{\mathrm{sp}}$$

considerado conocido (es la geometría observada, deformada).

### 1.3 Objetivo del Problema Inverso

El objetivo es reconstruir una configuración material $\Omega_{\mathrm{mat}}^*$ a partir de $\Omega$, mediante un nuevo mapeo:

$$\psi : \Omega \to \mathbb{R}^d$$

que se interpreta como una "deformación inversa" que lleva la configuración espacial comprimida hacia una configuración material reconstruida:

$$\Omega_{\mathrm{mat}}^* := \psi(\Omega)$$

Se elige la parametrización aditiva:

$$\psi(x) = x + u_{\mathrm{inv}}(x)$$

donde $u_{\mathrm{inv}} : \Omega \to \mathbb{R}^d$ es el campo de desplazamiento inverso.

**Ideal teórico:**

$$\psi \approx \varphi_{\mathrm{dir}}^{-1}, \quad \text{o bien} \quad \Omega_{\mathrm{mat}}^* \approx \Omega_{\mathrm{ref}}$$

en el sentido geométrico que se quiera (altura, forma global, distancia apropiada entre dominios, etc.).

---

## 2. Cinemática Inversa

### 2.1 Gradiente de Deformación Inverso

Dado el mapeo inverso $\psi(x) = x + u_{\mathrm{inv}}(x)$, se define el gradiente de deformación asociado como:

$$F_{\mathrm{inv}}(x) := \nabla_x \psi(x) = I + \nabla_x u_{\mathrm{inv}}(x)$$

donde $I$ es el tensor identidad en $\mathbb{R}^d$.

### 2.2 Determinante Jacobiano

Cambio de volumen asociado al mapeo inverso:

$$J_{\mathrm{inv}}(x) := \det F_{\mathrm{inv}}(x)$$

### 2.3 Tensor de Cauchy-Green Derecho

Tensor de Cauchy-Green derecho asociado al mapeo inverso:

$$C_{\mathrm{inv}}(x) := F_{\mathrm{inv}}(x)^{T} F_{\mathrm{inv}}(x)$$

### 2.4 Primer Invariante

Primer invariante de $C_{\mathrm{inv}}$:

$$I_c(x) := \operatorname{tr}\big( C_{\mathrm{inv}}(x) \big)$$

### 2.5 Interpretación Geométrica

Geométricamente, $F_{\mathrm{inv}}(x)$ aproxima $\partial X^*/\partial x$, donde $X^* = \psi(x)$ son las coordenadas "materiales reconstruidas".

Es decir, $F_{\mathrm{inv}}$ describe cómo se deforma la configuración espacial $\Omega$ para producir una nueva configuración material $\Omega_{\mathrm{mat}}^*$.

---

## 3. Modelo Constitutivo Hiperlástico (Neo-Hooke Inverso)

### 3.1 Densidad de Energía Desviadora

Se considera un material hiperlástico con densidad de energía (en forma desviadora):

$$\psi_{\mathrm{dev}}(F_{\mathrm{inv}}) := \frac{\mu}{2} \big( I_c - 2 - 2 \ln J_{\mathrm{inv}} \big)$$

donde $\mu > 0$ es el parámetro de Lamé de cortante.

### 3.2 Tensor de Piola-Kirchhoff Desviador

A partir de $\psi_{\mathrm{dev}}$ se define el tensor de Piola-Kirchhoff de primer tipo desviador mediante derivación:

$$P_{\mathrm{dev}}(x) := \frac{\partial \psi_{\mathrm{dev}}}{\partial F_{\mathrm{inv}}}(x)$$

### 3.3 Presión Inversa

Para tratar el régimen casi incompresible se introduce una presión escalar:

$$p_{\mathrm{inv}} : \Omega \to \mathbb{R}$$

que actúa como variable adicional (multiplicador de Lagrange asociado a la restricción volumétrica).

### 3.4 Tensor de Piola Total Inverso

El tensor de Piola total inverso se define como:

$$P_{\mathrm{inv}}(x) := P_{\mathrm{dev}}(x) + p_{\mathrm{inv}}(x)\,F_{\mathrm{inv}}(x)^{-T}$$

### 3.5 Relación Constitutiva Volumétrica

La incompresibilidad (o casi incompresibilidad) se modela mediante una relación constitutiva entre $J_{\mathrm{inv}}$ y $p_{\mathrm{inv}}$; por ejemplo, en forma penalizada:

$$\ln J_{\mathrm{inv}}(x) - \frac{p_{\mathrm{inv}}(x)}{\lambda} = 0$$

donde $\lambda > 0$ es el parámetro de Lamé volumétrico, grande en el límite casi incompresible.

---

## 4. Espacios Funcionales y Formulación Variacional Mixta

### 4.1 Descomposición de Frontera

Sea $\Gamma = \partial \Omega$ la frontera de la configuración espacial, con una descomposición en partes de Dirichlet y Neumann:

$$\Gamma = \Gamma_D \cup \Gamma_N, \quad \Gamma_D \cap \Gamma_N = \emptyset$$

En el contexto típico de compresión de una viga:

- $\Gamma_D$ incluye la cara inferior (base) y la cara superior (borde comprimido/expandido)
- $\Gamma_N$ incluye el resto de la frontera, donde no se prescriben tracciones (condiciones naturales)

### 4.2 Espacio de Desplazamiento Inverso

$$V_u := \{ v \in [H^1(\Omega)]^d : v = 0 \text{ en }\Gamma_D^{\mathrm{(fijada)}} \}$$

donde en $\Gamma_D$ se descompone la condición de Dirichlet en:
- Una parte fijada homogénea (por ejemplo, la base)
- Otra parte en la que se impone un desplazamiento no homogéneo (por ejemplo, expansión vertical en el tope)

### 4.3 Espacio de Presión

$$V_p := L^2(\Omega)$$

### 4.4 Espacio Mixto

$$V := V_u \times V_p$$

Las variables incógnitas son $(u_{\mathrm{inv}}, p_{\mathrm{inv}}) \in V$, y se consideran pares de prueba $(v, q) \in V$.

---

## 5. Problema Variacional Inverso

### 5.1 Ecuaciones de Equilibrio

En ausencia de fuerzas de cuerpo y tracciones exteriores (caso puramente geométrico-inverso), el equilibrio en la configuración espacial se expresa como:

$$\nabla_x \cdot P_{\mathrm{inv}}(x) = 0 \quad \text{en } \Omega$$

complementado con la ecuación volumétrica:

$$\ln J_{\mathrm{inv}}(x) - \frac{p_{\mathrm{inv}}(x)}{\lambda} = 0 \quad \text{en } \Omega$$

y condiciones de contorno apropiadas para $u_{\mathrm{inv}}$ en $\Gamma_D$.

### 5.2 Formulación Variacional Inversa

**Encontrar** $(u_{\mathrm{inv}}, p_{\mathrm{inv}}) \in V$ tales que, para todo $(v, q) \in V$:

$$\begin{aligned}
0 &= G_{\mathrm{inv}}(u_{\mathrm{inv}}, p_{\mathrm{inv}}; v, q) \\
&:= \int_{\Omega} P_{\mathrm{inv}}(u_{\mathrm{inv}},p_{\mathrm{inv}}) : \nabla v \, dx + \int_{\Omega} \big( \ln J_{\mathrm{inv}}(u_{\mathrm{inv}}) - p_{\mathrm{inv}}/\lambda \big)\, q \, dx
\end{aligned}$$

Esto define un **sistema no lineal acoplado** en $(u_{\mathrm{inv}}, p_{\mathrm{inv}})$.

### 5.3 Carácter "Inverso" del Problema

**Problema Directo:**
- Formulado en $\Omega_{\mathrm{ref}}$
- Determina un mapeo $\varphi_{\mathrm{dir}}$ bajo cargas conocidas

**Problema Inverso:**
- Formulado en $\Omega_{\mathrm{sp}}$
- Prescribe ciertas condiciones geométricas o de desplazamiento sobre $\Gamma$ (por ejemplo, una expansión en la cara superior)
- Busca un mapeo $\psi$ que "revierte" la deformación producida en el directo

**Observación importante:** No se introduce explícitamente el conocimiento de $\varphi_{\mathrm{dir}}$ en la formulación; en su lugar, se plantea un problema de equilibrio hiperlástico en la configuración espacial, pero con condiciones de borde y restricciones diseñadas para que la imagen de $\Omega$ bajo $\psi$ se aproxime a $\Omega_{\mathrm{ref}}$.

### 5.4 Formulación Abstracta

En términos formales, el problema inverso puede verse como:

$$\begin{cases}
\text{Encontrar } (u_{\mathrm{inv}}, p_{\mathrm{inv}}) \in V \text{ tal que}\\
G_{\mathrm{inv}}(u_{\mathrm{inv}}, p_{\mathrm{inv}}; v, q) = 0, \quad \forall (v,q)\in V\\
\psi(x) = x + u_{\mathrm{inv}}(x) \text{ verifica una condición geométrica objetivo,}\\
\hspace{1cm}\text{por ejemplo } \mathcal{M}(\psi(\Omega)) = \mathcal{M}(\Omega_{\mathrm{ref}})
\end{cases}$$

donde $\mathcal{M}$ es una medida geométrica de interés (altura, volumen, etc.).

---

## 6. Condiciones de Contorno como Restricción Geométrica Inversa

### 6.1 Definición de Alturas

Sea:

$$H_{\mathrm{ref}} := \sup_{X \in \Omega_{\mathrm{ref}}} X_2 - \inf_{X \in \Omega_{\mathrm{ref}}} X_2$$

$$H_{\mathrm{sp}} := \sup_{x \in \Omega} x_2 - \inf_{x \in \Omega} x_2$$

Se desea que la configuración reconstruida $\Omega_{\mathrm{mat}}^* = \psi(\Omega)$ tenga altura:

$$H_{\mathrm{mat}}^* := \sup_{X^* \in \Omega_{\mathrm{mat}}^*} X^*_2 - \inf_{X^* \in \Omega_{\mathrm{mat}}^*} X^*_2$$

satisfaciendo:

$$H_{\mathrm{mat}}^* \approx H_{\mathrm{ref}}$$

### 6.2 Imposición de Expansión Objetivo

Una forma natural de imponer esto variacionalmente es prescribir en la cara superior $\Gamma_{\text{top}} \subset \Gamma$ un desplazamiento vertical no homogéneo $g(t)$ tal que la expansión total:

$$\delta_{\max} := H_{\mathrm{ref}} - H_{\mathrm{sp}}$$

sea alcanzada. De modo que, al final del proceso, la altura de $\Omega_{\mathrm{mat}}^*$ coincida con la original.

### 6.3 Condiciones de Borde Explícitas

En forma abstracta:

$$u_{\mathrm{inv},2}(x) = g(x) = \delta_{\max} \quad \text{en } \Gamma_{\text{top}}$$

$$u_{\mathrm{inv}}(x) = 0 \quad \text{en } \Gamma_{\text{bottom}}$$

y eventualmente $u_{\mathrm{inv},1}$ libre en $\Gamma_{\text{top}}$ para permitir deslizamiento tangencial.

### 6.4 Síntesis de Restricciones

Así, el problema inverso combina:

1. Una ecuación de equilibrio hiperlástico en la configuración espacial
2. Una condición volumétrica casi incompresible
3. Condiciones de borde que implementan explícitamente la expansión necesaria para revertir la compresión observada en el problema directo

---

## 7. Reconstrucción Geométrica

### 7.1 Mapeo Reconstructor

Una vez resuelto el problema inverso, se obtiene un par $(u_{\mathrm{inv}}, p_{\mathrm{inv}})$. El mapeo:

$$\psi(x) = x + u_{\mathrm{inv}}(x)$$

genera la configuración material reconstruida:

$$\Omega_{\mathrm{mat}}^* := \psi(\Omega)$$

### 7.2 Métricas de Error de Reconstrucción

El error de reconstrucción puede cuantificarse, por ejemplo, mediante una métrica geométrica:

**Error de altura:**

$$e_H := |H_{\mathrm{ref}} - H_{\mathrm{mat}}^*|$$

**Error relativo:**

$$e_{\mathrm{rel}} := \frac{e_H}{H_{\mathrm{ref}}} \times 100\%$$

### 7.3 Condición de Buen Planteamiento

Desde el punto de vista formal, un buen problema inverso está planteado de modo que, para las condiciones de borde elegidas y el modelo constitutivo dado, se obtenga:

$$e_{\mathrm{rel}} \ll 1$$

y, más idealmente:

$$\Omega_{\mathrm{mat}}^* \approx \Omega_{\mathrm{ref}}$$

en una métrica de dominios adecuada.

---

## 8. Conclusión

La **formulación inversa** es un **problema variacional mixto desplazamiento-presión** planteado en la configuración espacial $\Omega_{\mathrm{sp}}$, cuyo propósito es encontrar un mapeo hiperlástico:

$$\psi(x) = x + u_{\mathrm{inv}}(x)$$

que, bajo condiciones de contorno y restricciones volumétricas apropiadas, reconstruya una configuración material $\Omega_{\mathrm{mat}}^*$ que reproduzca, en la medida de lo posible, la configuración de referencia $\Omega_{\mathrm{ref}}$.

### Características Clave

- **Formulación variacional mixta:** desplazamientos + presión
- **Dominio de cálculo:** configuración espacial (deformada)
- **Incógnitas:** desplazamiento inverso y presión
- **Objetivo:** revertir la deformación directa
- **Constraint:** restricción geométrica (recuperar altura original)
- **Modelo:** Neo-Hooke casi incompresible